In [19]:
import sys
import os
#sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('"Top 10 Funds by AUM Positioning - Data Gen.ipynb"')))))
#sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('"Top 10 Funds by AUM Positioning - Data Gen.ipynb"')))))
import apis.pa.mypaengine as pa
import apis.qe.myqengine as qe

In [20]:
time_series = qe.TimeSeries(start_date = '0M',end_date = '0M')

univ = qe.ScreenUniverse(universe_expr = '(ISAV(FFD_SHR_CLS_AUM(0,ACT,ROLL,USD))=1 AND ISAV(P_PRICE(0,USD))=1 AND UPPERCASE(P_EXCOUNTRY(ISO2))=UPPERCASE("US") AND PROPER(FFD_CLASS_ASSET(TEXT))=PROPER("EQUITY"))=1')

req = qe.calculate(universe = univ, dates = time_series,formulas = ['P_PRICE(0,USD)',"FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)","LSS_TICKER","FFD_BMK_ID"])
df = req.data

In [21]:
df = req.data
df = df.rename(columns = {'P_PRICE(0,USD)':'price',
                       "FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)":'aum',
                       "LSS_TICKER":'lionshares_id',
                       "FFD_BMK_ID":'bench_id'})

t10_data ={}
bench_list = ['SP50','R.1000','R.1000G','R.1000V','R.2000','R.2000V','R.2000G','R.3000','R.MID','990300','892400','990100','899901','891800']
for bench in bench_list:
    t10_data[bench] = df[df['bench_id']==bench].dropna().sort_values('aum',ascending=False).head(10)

t10_data['SP50']

,DATE,UNIVERSE,price,aum,lionshares_id,bench_id
2127,20230731,SWPPX,70.88,7.506143e+10,SWPPX-US,SP50
8465,20230731,TRJZX,85.45,2.139821e+10,TRJZX-US,SP50
7158,20230731,HDGFX,31.69,1.632903e+10,HDGFX-US,SP50
5392,20230731,JUEMX,20.64,1.404336e+10,JUEMX-US,SP50
6452,20230731,PRUFX,84.72,1.350300e+10,PRUFX-US,SP50
5371,20230731,JHEQX,28.26,1.327973e+10,JHEQX-US,SP50
8456,20230731,TRHZX,120.76,1.295726e+10,TRHZX-US,SP50
1924,20230731,PRDGX,69.92,1.224666e+10,PRDGX-US,SP50
532,20230731,DFUSX,31.30,1.093649e+10,DFUSX-US,SP50
7182,20230731,HGIFX,44.98,9.912087e+09,HGIFX-US,SP50


In [22]:
pa_document_name = "PERSONAL:API_SP"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )


In [23]:

import pandas as pd
#Define PA Inputs
temp_df =t10_data['SP50']
temp_df['pa_port_id'] = "LION:"+temp_df['lionshares_id']
temp_df['pa_bench_id'] = "BENCH:"+temp_df['bench_id']

portfolios = temp_df['pa_port_id'].to_list()
benchmarks = temp_df['pa_bench_id'].to_list()
enddate = '0M'
pa_doc.run_calc(
               portfolios,
               benchmarks,
               end_date = enddate,
               curr = 'USD',
               mode= 'B&H'
              )
pa_doc.data[list(pa_doc.data.keys())[0]][0]

df_full = pd.DataFrame()
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_x_bench'] = x
    df_full = pd.concat([df_full,temp])

In [24]:
pa_document_name = "PERSONAL:API_RUSSELL"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )

for bench in ['R.1000','R.1000G','R.1000V','R.2000','R.2000V','R.2000G','R.3000','R.MID']:
    #Define PA Inputs
    temp_df =t10_data[bench]
    temp_df['pa_port_id'] = "LION:"+temp_df['lionshares_id']
    temp_df['pa_bench_id'] = "BENCH:"+temp_df['bench_id']

    portfolios = temp_df['pa_port_id'].to_list()
    benchmarks = temp_df['pa_bench_id'].to_list()
    enddate = '0M'
    pa_doc.run_calc(
                portfolios,
                benchmarks,
                end_date = enddate,
                curr = 'USD',
                mode= 'B&H'
                )
    for x in list(pa_doc.data.keys()):
        temp = pa_doc.data[x][0]
        temp['port_x_bench'] = x
        df_full = pd.concat([df_full,temp])
    #filepath = '../../data/t10_'+bench+'_weights_20230830.csv'
    #df_full.to_csv(filepath)
        

In [25]:
pa_document_name = "PERSONAL:API_MSCI"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )

for bench in ['990300','892400','990100','899901','891800']:
    #Define PA Inputs
    temp_df =t10_data[bench]
    temp_df['pa_port_id'] = "LION:"+temp_df['lionshares_id']
    temp_df['pa_bench_id'] = "BENCH:"+temp_df['bench_id']

    portfolios = temp_df['pa_port_id'].to_list()
    benchmarks = temp_df['pa_bench_id'].to_list()
    enddate = '0M'
    pa_doc.run_calc(
                portfolios,
                benchmarks,
                end_date = enddate,
                curr = 'USD',
                mode= 'B&H'
                )
    for x in list(pa_doc.data.keys()):
        temp = pa_doc.data[x][0]
        temp['port_x_bench'] = x
        df_full = pd.concat([df_full,temp])
        

In [26]:
df_full.head()

,Symbol,security0,Ticker-Region,GICS Sector,GICS Industry Name,GICS Industry Group,GICS Sub Industry,Region of Domicile,MSCI Country,P_Country,FF_Country,Price,Port.+Shares,Port.+Weight,Bench.+Shares,Bench.+Weight,Difference,Asset Type+(Client Definition/+FactSet),Asset Class+(Client Definition/+FactSet),port_x_bench
0,88579Y101,3M Company,MMM-US,Industrials,Industrial Conglomerates,Capital Goods,Industrial Conglomerates,North America,United States,US,UNITED STATES,111.5,1075142.0,0.159586,551.672,0.160497,-0.000911,Equity Common,Equity,LION:SWPPX-US_x_BENCH:SP50
1,831865209,A. O. Smith Corporation,AOS-US,Industrials,Building Products,Capital Goods,Building Products,North America,United States,US,UNITED STATES,72.629997,244991.0,0.023688,124.538,0.023601,0.000087,Equity Common,Equity,LION:SWPPX-US_x_BENCH:SP50
2,002824100,Abbott Laboratories,ABT-US,Health Care,Health Care Equipment & Supplies,Health Care Equipment & Services,Health Care Equipment,North America,United States,US,UNITED STATES,111.330002,3393718.0,0.50297,1738.947,0.505137,-0.002167,Equity Common,Equity,LION:SWPPX-US_x_BENCH:SP50
3,00287Y109,"AbbVie, Inc.",ABBV-US,Health Care,Biotechnology,Pharmaceuticals Biotechnology & Life Sciences,Biotechnology,North America,United States,US,UNITED STATES,149.580002,3443178.0,0.685626,1764.29,0.68858,-0.002954,Equity Common,Equity,LION:SWPPX-US_x_BENCH:SP50
4,B4BNMY3,Accenture Plc Class A,ACN-US,Information Technology,IT Services,Software & Services,It Consulting & Other Services,North America,United States,US,IRELAND,316.350006,1232494.0,0.519047,631.532,0.521284,-0.002236,Equity Common,Equity,LION:SWPPX-US_x_BENCH:SP50


In [31]:
df_full.to_pickle('../../data/t10_full_universe_weights_20230731.pkl')